In [ ]:
import os
import json
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

# Define metrics to extract
metrics = [
    "average_turnaroundTime",
    "average_responseTime",
    "average_fidelity",
    "sampling_overhead",
    "average_throughput",
    "average_utilization",
    "scheduler_latency",
    "makespan"
]

# Define folders for the datasets
folder_paths = {
    "FFD": os.path.join("component", "finalResult", "5_5", "FFD", "ghz"),
    "MILQ": os.path.join("component", "finalResult", "5_5", "MILQ", "ghz"),
    "NoTODS": os.path.join("component", "finalResult", "5_5", "NoTaDS", "ghz"),
    "MTMC": os.path.join("component", "finalResult", "5_5", "MTMC", "ghz"),
}

# Output base folder
output_base = os.path.join("analyze")
os.makedirs(output_base, exist_ok=True)

for num_circuit in range(2, 20):  # Loop through num_qubits values from 2 to 19
    pattern = re.compile(rf"{num_circuit}_(\d+)\.0_0\.json")

    data = []
    for dataset_name, folder_path in folder_paths.items():
        for file in os.listdir(folder_path):
            if pattern.match(file):
                with open(os.path.join(folder_path, file), "r") as f:
                    content = json.load(f)
                    entry = {metric: content.get(metric, None) for metric in metrics}
                    entry["filename"] = file
                    entry["dataset"] = dataset_name
                    data.append(entry)

    if not data:
        print(f"No matching files found for num_circuit={num_circuit}.")
        continue

    df = pd.DataFrame(data)
    df["num_qubits_each_circuit"] = df["filename"].str.extract(rf"{num_circuit}_(\d+)\.0_0\.json")[0].astype(int)
    df = df.sort_values(by=["num_qubits_each_circuit", "dataset"])

    # Create output folder for this circuit size
    circuit_output_dir = os.path.join(output_base, f"{num_circuit}_circuits")
    os.makedirs(circuit_output_dir, exist_ok=True)

    for metric in metrics:
        plt.figure(figsize=(8, 5))
        sns.lineplot(data=df, x="num_qubits_each_circuit", y=metric, hue="dataset", marker="o")
        plt.title(f"{metric} (num_circuit={num_circuit})")
        plt.xlabel("Num Qubits per Batch")
        plt.ylabel(metric)
        plt.xticks(rotation=45)
        plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        plt.legend(title="Dataset")
        plt.tight_layout()

        # Save plot
        plot_dir = os.path.join(circuit_output_dir, "plot")
        os.makedirs(plot_dir, exist_ok=True)
        plot_path = os.path.join(plot_dir, f"{metric}_plot.pdf")
        plot_index = 1
        orig_plot_path = plot_path
        while os.path.exists(plot_path):
            plot_path = os.path.join(plot_dir, f"{metric}_plot_{plot_index}.pdf")
            plot_index += 1
        plt.savefig(plot_path)
        plt.close()

        # Save CSV
        csv_dir = os.path.join(circuit_output_dir, "csv")
        os.makedirs(csv_dir, exist_ok=True)
        csv_path = os.path.join(csv_dir, f"{metric}_data.csv")
        csv_index = 1
        orig_csv_path = csv_path
        while os.path.exists(csv_path):
            csv_path = os.path.join(csv_dir, f"{metric}_data_{csv_index}.csv")
            csv_index += 1
        metric_df = df[["num_qubits_each_circuit", "dataset", metric]].dropna()
        metric_df.to_csv(csv_path, index=False)

    print(f"Saved plots and CSVs for num_circuit={num_circuit} to: {circuit_output_dir}")

Saved plots and CSVs for num_circuit=2 to: analyze/2_circuits
Saved plots and CSVs for num_circuit=3 to: analyze/3_circuits
Saved plots and CSVs for num_circuit=4 to: analyze/4_circuits
Saved plots and CSVs for num_circuit=5 to: analyze/5_circuits
Saved plots and CSVs for num_circuit=6 to: analyze/6_circuits
Saved plots and CSVs for num_circuit=7 to: analyze/7_circuits
Saved plots and CSVs for num_circuit=8 to: analyze/8_circuits
Saved plots and CSVs for num_circuit=9 to: analyze/9_circuits
Saved plots and CSVs for num_circuit=10 to: analyze/10_circuits
Saved plots and CSVs for num_circuit=11 to: analyze/11_circuits
Saved plots and CSVs for num_circuit=12 to: analyze/12_circuits
Saved plots and CSVs for num_circuit=13 to: analyze/13_circuits
Saved plots and CSVs for num_circuit=14 to: analyze/14_circuits
Saved plots and CSVs for num_circuit=15 to: analyze/15_circuits
Saved plots and CSVs for num_circuit=16 to: analyze/16_circuits
Saved plots and CSVs for num_circuit=17 to: analyze/17_c

In [ ]:
import os
import json
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

# Define metrics to extract
metrics = [
    "average_turnaroundTime",
    "average_responseTime",
    "average_fidelity",
    "sampling_overhead",
    "average_throughput",
    "average_utilization",
    "time_generation",
    "makespan"
]

# Define folders for the datasets
folder_paths = {
    "FFD": os.path.join("component", "finalResult", "5_5", "FFD", "ghz"),
    "MILQ_extend": os.path.join("component", "finalResult", "5_5", "MILQ_extend", "ghz"),
    "NoTODS": os.path.join("component", "finalResult", "5_5", "NoTODS", "ghz"),
    "MTMC": os.path.join("component", "finalResult", "5_5", "MTMC", "ghz"),
}

# Output base folder
output_base = os.path.join("component", "finalResult", "code-3")
os.makedirs(output_base, exist_ok=True)

for num_qubit in range(2, 11):
    pattern = re.compile(rf"(\d+)_{num_qubit}\.0_0\.json")

    data = []
    for dataset_name, folder_path in folder_paths.items():
        for file in os.listdir(folder_path):
            match = pattern.match(file)
            if match:
                num_circuit = int(match.group(1))
                with open(os.path.join(folder_path, file), "r") as f:
                    content = json.load(f)
                    entry = {metric: content.get(metric, None) for metric in metrics}
                    entry["filename"] = file
                    entry["dataset"] = dataset_name
                    entry["num_circuit"] = num_circuit
                    data.append(entry)

    if not data:
        print(f"No matching files found for num_qubit={num_qubit}.")
        continue

    df = pd.DataFrame(data)
    df = df.sort_values(by=["num_circuit", "dataset"])

    # Output folder for current num_qubit
    qubit_output_dir = os.path.join(output_base, f"{num_qubit}_qubits")
    os.makedirs(qubit_output_dir, exist_ok=True)

    for metric in metrics:
        plt.figure(figsize=(8, 5))
        sns.lineplot(
            data=df,
            x="num_circuit",
            y=metric,
            hue="dataset",
            style="dataset",
            marker="o",
            markersize=8,
            linewidth=2,
            palette="tab10"
        )
        plt.title(f"{metric} (num_qubit={num_qubit})")
        plt.xlabel("Num Circuits")
        plt.ylabel(metric)
        plt.xticks(rotation=45)
        plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        plt.legend(title="Dataset", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()

        # Save the figure
        plot_path = os.path.join(qubit_output_dir, f"{metric}_plot.pdf")
        plt.savefig(plot_path)
        plt.close()

        # Save plot data to CSV
        metric_df = df[["num_circuit", "dataset", metric]].dropna()
        csv_path = os.path.join(qubit_output_dir, f"{metric}_data.csv")
        metric_df.to_csv(csv_path, index=False)

    print(f"Saved plots and CSVs for num_qubit={num_qubit} to: {qubit_output_dir}")

In [ ]:
import os
import json
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

# Define metrics to extract
metrics = [
    "average_turnaroundTime",
    "average_responseTime",
    "average_fidelity",
    "sampling_overhead",
    "average_throughput",
    "average_utilization",
    "time_generation",
    "makespan"
]

# Define folders for the datasets
folder_paths = {
    "FFD": os.path.join("component", "finalResult", "5_5", "FFD", "ghz"),
    "MILQ_extend": os.path.join("component", "finalResult", "5_5", "MILQ_extend", "ghz"),
    "NoTODS": os.path.join("component", "finalResult", "5_5", "NoTODS", "ghz"),
    "MTMC": os.path.join("component", "finalResult", "5_5", "MTMC", "ghz"),
}

# Compile regex to match files like 2_0.0_0.json → 10_6.0_0.json
pattern = re.compile(r"(\d+)_(\d+)\.0_0\.json")

# Read data from all datasets
data = []
for dataset_name, folder_path in folder_paths.items():
    for file in os.listdir(folder_path):
        match = pattern.match(file)
        if match:
            num_circuit = int(match.group(1))
            num_qubits_each_circuit = int(match.group(2))
            with open(os.path.join(folder_path, file), "r") as f:
                content = json.load(f)
                entry = {metric: content.get(metric, None) for metric in metrics}
                entry["filename"] = file
                entry["dataset"] = dataset_name
                entry["num_circuit"] = num_circuit
                entry["num_qubits_each_circuit"] = num_qubits_each_circuit
                data.append(entry)

if not data:
    print(f"No matching files found. Please check folders and regex pattern.")
    exit()

# Convert to DataFrame
df = pd.DataFrame(data)
df = df.sort_values(by=["num_circuit", "num_qubits_each_circuit", "dataset"])

# Output folder
output_dir = os.path.join("component", "finalResult", "code-5")
os.makedirs(output_dir, exist_ok=True)

# Generate plot and CSV for each metric
for metric in metrics:
    plt.figure(figsize=(8, 5))
    sns.lineplot(
        data=df,
        x="num_circuit",
        y=metric,
        hue="dataset",
        style="num_qubits_each_circuit",
        markers=True,
        dashes=True,
        palette="tab10",
        linewidth=2,
        markersize=8
    )
    plt.title(metric)
    plt.xlabel("Num Circuits")
    plt.ylabel(metric)
    plt.xticks(rotation=45)
    plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    plt.legend(title="Dataset & NumQubits", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    # Save plot
    pdf_path = os.path.join(output_dir, f"{metric}_plot.pdf")
    plt.savefig(pdf_path)
    plt.close()

    # Save data used for this metric to CSV
    csv_data = df[["num_circuit", "num_qubits_each_circuit", "dataset", metric]].dropna()
    csv_path = os.path.join(output_dir, f"{metric}_data.csv")
    csv_data.to_csv(csv_path, index=False)

    print(f"Saved: {pdf_path} and {csv_path}")